# 2-3. 특성공학(Feature Engineering)

- 수식화(회귀분석) -> 기계학습(데이터 마이닝))
- **특성공학(Feature Engineering)**



#### 5. Imbalanced Data Sampling
- 비율이 다른 범주형 Y에 대해 분류를 진행할 때 , 데이터의 비율을 맞춰주는 작업
- 학습하는 과정에서학습데이터에 적용하여 사용 -> Pipe Line(sklearn)

- 1) Under Sampling:  데이터의 비율이 적은 쪽으로 데이터를 맞춰주는 작업
    - Random Under Sampling: 데이터의 비율이 많은 쪽에 값을 무작위로 줄이는 방법
    - Tomeke's Link: 인접합 Class (서로 다른 범주형 항목)들을 뭈어 밀집되어 있는 부분을 데이터를 제거
    - CNN(): 비율이 많은 쪽에데이터에서 밀도가 높은 데이터를 지움
    - Onse
    
    
- 2) Over Sampling: 데이터의 비율이 큰 쪽으로 데이터를 생성
    - Random Over Sampling: 무작위하게 데이터의 비율이 많은 수에 맞춰 생성
    - SMOTE(Synthetic Minority over Sampling Techique): KNN기법을 활용해, 비율이 적은 쪽 데이터를 K개의 인접 데이터 수 만큼 묶어, 묶인 데이터 내 중심 위치를 찬아 새로운 데이터 생성
    - ADASYN(Adaptive Synthetic Sampling): SMOTE 기법을 이용해 데이터를 생성하는 단계에서 임의의 작은 값을 (감마 확률 밀도 함수) 더하여 사실적 데이터로 생성
    
-------------------------------------------

In [1]:
import numpy as np
import pandas as pd # 정형데이터 전처리 / 통계 연산 수행

import seaborn as sns # 통계적 시각화 연산
import matplotlib as mpl # 시각화 옵션 (외부)
import matplotlib.pyplot as plt # 시각화 옵션 (내부)
import plotly.express as px # 동적 그래프 시각화
import scipy.stats as stats # 통계적 가설 검정 수행

mpl.rc('font',family='Malgun Gothic') # 한글 글꼴 설정

In [2]:
df1 = pd.read_csv('01_Data.csv')
df1

,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,Customer_Type,Age,Address1,Address2,State,Overdue_count,Overdue_Type,Gender,Credit_Rank,Bank
0,1,66758234,렌탈,일반계약,영업방판,2019-05-06,60,CMS,DES-1,96900,개인,42.0,경기도,경기도,계약확정,0,없음,여자,9.0,새마을금고
1,2,66755948,렌탈,교체계약,영업방판,2020-02-20,60,카드이체,DES-1,102900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,2.0,현대카드
2,3,66756657,렌탈,일반계약,홈쇼핑/방송,2019-02-28,60,CMS,DES-1,96900,개인,48.0,경기도,경기도,계약확정,0,없음,여자,8.0,우리은행
3,4,66423450,멤버십,멤버십3유형,재계약,2019-05-13,12,CMS,DES-1,66900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,5.0,농협회원조합
4,5,66423204,멤버십,멤버십3유형,재계약,2019-05-10,12,CMS,DES-1,66900,개인,60.0,경기도,경기도,기간만료,12,있음,남자,8.0,농협회원조합
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51296,51298,66579515,렌탈,프로모션계약,대형마트A,2019-03-01,60,CMS,DES-3A,96900,개인,47.0,경기도,경기도,계약확정,0,없음,남자,NaN,기업은행
51297,51299,66799558,렌탈,일반계약,대형마트A,2019-04-01,60,CMS,DES-1,96900,개인,42.0,경기도,경기도,계약확정,0,없음,여자,8.0,새마을금고
51298,51300,66799197,렌탈,프로모션계약,영업방판,2019-04-01,39,카드이체,ERA,120900,개인,65.0,서울특별시,서울특별시,계약확정,0,없음,여자,1.0,롯데카드
51299,51301,66792778,렌탈,일반계약,홈쇼핑/방송,2020-02-06,60,카드이체,DES-1,96900,개인,54.0,서울특별시,서울특별시,계약확정,0,없음,여자,2.0,롯데카드


In [3]:
# 해약 여부를 Target으로 만들거임


df1['Target'] = df1['State'].replace({'계약확정': '정상', '기간만료': '정상',
                     '해약확정': '해약', '해약진행중': '해약'})

In [4]:
df1.columns

Index(['Index', 'Member_ID', 'Sales_Type', 'Contract_Type', 'Channel',
       'Datetime', 'Term', 'Payment_Type', 'Product_Type', 'Amount_Month',
       'Customer_Type', 'Age', 'Address1', 'Address2', 'State',
       'Overdue_count', 'Overdue_Type', 'Gender', 'Credit_Rank', 'Bank',
       'Target'],
      dtype='object')

In [5]:
X = df1[['Sales_Type', 'Term', 'Product_Type',
            'Amount_Month', 'Age', 'Gender','Credit_Rank']]

Y = df1['Target']

In [6]:
from sklearn.model_selection import train_test_split # 머신러닝 데이터 split
from imblearn.pipeline import make_pipeline  # 학습 + 특성공학 파이프 구축
from sklearn.compose import make_column_transformer  # 데이터 타입별 특성 공학
from sklearn.impute import SimpleImputer # 결측값 대치
from sklearn.preprocessing import MinMaxScaler # 숫자 데이터 스케일링
from sklearn.preprocessing import OneHotEncoder  # 문자 데이터 인코딩
from sklearn.model_selection import GridSearchCV  # 교차검증 + 매개변수 튜닝
from sklearn.tree import DecisionTreeClassifier  # 학습수행 알고리즘
from sklearn.metrics import classification_report # 분류모델의 성능 측정함수
from sklearn.metrics import accuracy_score # 머신러닝 평가
import pickle   # 학습 모델을 파일로 저장

In [7]:
X_train, X_test,Y_train, Y_test = train_test_split(X, Y, random_state=1234)

In [8]:
numeric_pipe = make_pipeline((SimpleImputer(strategy='mean')),
                              (MinMaxScaler()))
                             
category_pipe = make_pipeline((SimpleImputer(strategy='most_frequent')),
                              (OneHotEncoder()))

In [9]:
numeric_list = X.describe().columns.tolist()
category_list = X.describe(include='object').columns.tolist()

preprocessing_pipe = make_column_transformer((numeric_pipe, numeric_list),
                                             (category_pipe, category_list))

model_pipe = make_pipeline(preprocessing_pipe, DecisionTreeClassifier())

In [10]:
# max_depth 값을 5부터 9까지 번갈아가며 대입
hyperparameter_dict = {'decisiontreeclassifier__max_depth': range(5,10)}

grid_model = GridSearchCV(model_pipe, cv=5,
                          param_grid= hyperparameter_dict, scoring ='f1')
                                # 성능 좌표를  정확도가이닌 'f1'으로 설정
    
grid_model.fit(X_train, Y_train)

C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1261, in fbeta_score
    _, _, f, _ = precision_recall_

C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1261, in fbeta_score
    _, _, f, _ = precision_recall_

C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1261, in fbeta_score
    _, _, f, _ = precision_recall_

C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1261, in fbeta_score
    _, _, f, _ = precision_recall_

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('minmaxscaler',
                                                                                          MinMaxScaler())]),
                                                                         ['Term',
                                                                          'Amount_Month',
                                                                          'Age',
                                                                          'Credit_Rank']),
                                                           

In [11]:
best_model = grid_model.best_estimator_

In [12]:
def evalutation_func(best_model):
    Y_train_pred = best_model.predict(X_train)
    Y_test_pred = best_model.predict(X_test)
    print('학습 성능')
    print(classification_report(Y_train, Y_train_pred))
    print('일반화 성능')
    print(classification_report(Y_test, Y_test_pred))

In [13]:
evalutation_func(best_model)

학습 성능
              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     37984
          해약       0.80      0.01      0.02       491

    accuracy                           0.99     38475
   macro avg       0.89      0.50      0.50     38475
weighted avg       0.98      0.99      0.98     38475

일반화 성능
              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     12681
          해약       0.00      0.00      0.00       145

    accuracy                           0.99     12826
   macro avg       0.49      0.50      0.50     12826
weighted avg       0.98      0.99      0.98     12826



# Imbalanced Data Sampling

## Under Sampling

In [14]:
from imblearn.under_sampling import RandomUnderSampler

In [15]:
under_sampler = RandomUnderSampler()
under_sampler.fit(X,Y)

RandomUnderSampler()

In [16]:
X_under, Y_under = under_sampler.fit_resample(X, Y)

In [17]:
Y_under.value_counts()

정상    636
해약    636
Name: Target, dtype: int64

In [18]:
# TomekLinks

from imblearn.under_sampling import TomekLinks

In [19]:
df1_clean = df1.dropna()

In [20]:
under_sampler = TomekLinks()
under_sampler.fit(df1_clean[['Age', 'Term']], df1_clean[['Target']])

TomekLinks()

In [21]:
X_under, Y_under = under_sampler.fit_resample(df1_clean[['Age', 'Term']],
                                              df1_clean[['Target']])
Y_under.value_counts()

Target
정상        40137
해약          510
dtype: int64

In [22]:
from imblearn.over_sampling import SMOTE

In [32]:
# 모델 적용시
model_pipe = make_pipeline(preprocessing_pipe, SMOTE(), DecisionTreeClassifier())

#'decisiontreeclassifier__min_samples_leaf': range(5,15),
                      #'decisiontreeclassifier__min_samples_split': range(5,15)

In [38]:
# max_depth 값을 5부터 9까지 번갈아가며 대입
hyperparameter_dict = {'decisiontreeclassifier__max_depth': range(5,10),
                       'decisiontreeclassifier__min_samples_leaf': range(5,10),
                       'decisiontreeclassifier__min_samples_split': range(5,10)}

grid_model = GridSearchCV(model_pipe, cv=5,
                          param_grid= hyperparameter_dict,
                          scoring ='f1',
                         n_jobs=-1)
                                # 성능 좌표를  정확도가이닌 'f1'으로 설정
                            
    
grid_model.fit(X_train, Y_train)

C:\Users\DMCCONET\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('minmaxscaler',
                                                                                          MinMaxScaler())]),
                                                                         ['Term',
                                                                          'Amount_Month',
                                                                          'Age',
                                                                          'Credit_Rank']),
                                                           

In [39]:
best_model = grid_model.best_estimator_

In [40]:
evalutation_func(best_model)

학습 성능
              precision    recall  f1-score   support

          정상       0.99      0.95      0.97     37984
          해약       0.09      0.38      0.15       491

    accuracy                           0.94     38475
   macro avg       0.54      0.67      0.56     38475
weighted avg       0.98      0.94      0.96     38475

일반화 성능
              precision    recall  f1-score   support

          정상       0.99      0.95      0.97     12681
          해약       0.07      0.34      0.12       145

    accuracy                           0.94     12826
   macro avg       0.53      0.64      0.55     12826
weighted avg       0.98      0.94      0.96     12826

